![imagenes](logo.png)

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7, 7)

### Ejemplo 1: Análisis de sentimiento en texto

Las RNN funciona de forma particularmente bien a la hora de tratar con texto, ya que por definición el texto es una secuencia ordenada de palabras, y las RNN están especializadas en información secuencial.

En este ejemplo vamos a hacer un modelo que haga **Análisis de Sentimiento**. Estos análisis consisten en dado un texto, predecir si el texto es un texto con tono positivo o negativo.

Para ello vamos a usar un dataset de entrenamiento creado por la Sociedad Española del Procesado de Lenguaje Natural (SEPLN), y que usan en una conferencia llamada TASS (Taller de Análisis Semántico en la SEPLN).

### Cargamos los datos

In [ ]:
import pandas as pd
pd.set_option('max_colwidth',1000)

In [ ]:
tweets_corpus = pd.read_csv("texto_extendido.csv",encoding = "latin-1")

In [ ]:
tweets_corpus.head(10)

In [ ]:
tweets_corpus.shape

In [ ]:
tweets_corpus = tweets_corpus.rename(columns={"polarity":"value"})

In [ ]:
tweets_corpus.columns

In [ ]:
tweets_corpus = tweets_corpus.astype("str")

In [ ]:
tweets_corpus.value.value_counts()

In [ ]:
polaridades = pd.get_dummies(tweets_corpus.value)
polaridades.head()

In [ ]:
polaridades_one_hot = polaridades.values
polaridades_one_hot[:5]

In [ ]:
polaridades_dict = dict(zip(range(6), polaridades.columns))
polaridades_dict

### Procesamos el texto

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Vamos a tokenizar los tweets. Vamos a convertirlos en vectores de números, donde cada número representa una palabra. Usaremos las 1500 palabrás más comunes en el dataset

In [ ]:
max_palabras = 1500
tokenizer = Tokenizer(num_words=max_palabras)
tokenizer.fit_on_texts(tweets_corpus.content.values)
X = tokenizer.texts_to_sequences(tweets_corpus.content.values)
X = pad_sequences(X)

In [ ]:
X.shape

In [ ]:
X[0]

Vemos que el tweet más largo tiene 32 palabras.

In [ ]:
tokenizer.word_index


Podemos crear un mapa inverso de la lista de palabras, esto nos permite reconstruir un tweet en función de su versión vectorizada

In [ ]:
mapa_inverso = dict(map(reversed, tokenizer.word_index.items()))

In [ ]:
mapa_inverso

Así si tomamos el primer tweet

In [ ]:
tweets_corpus.content.values[0]

Vemos su versión vectorizada:

In [ ]:
X[0]

Podemos reconstruirlo con el mapa_inverso del tokenizador

In [ ]:
[mapa_inverso[i] for i in X[0] if i in mapa_inverso]

### Creamos la RNN

Como capa de entrada vamos a usar la capa [`Embedding`](https://keras.io/layers/embeddings/). Es una capa que se encarga de transformar una matriz de texto (donde cada número representa una palabra), en una matriz que representa las relaciones entre las frases  en función de sus palabras.

Digamos que un Embedding (word embedding) es una forma mejor de representar texto (captura mejor la información).

In [ ]:
from keras import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU

numero_clases = 6

modelo_sentimiento = Sequential()
modelo_sentimiento.add(Embedding(max_palabras, 128, input_length=X.shape[1]))
modelo_sentimiento.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
modelo_sentimiento.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
#modelo_sentimiento.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))

modelo_sentimiento.add(Dense(numero_clases, activation='softmax'))

In [ ]:
modelo_sentimiento.compile(loss = 'categorical_crossentropy', optimizer='adam',
                           metrics = ['accuracy'])
print(modelo_sentimiento.summary())

In [ ]:
from sklearn.model_selection import train_test_split

Y = polaridades_one_hot
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20)

In [ ]:
from keras.callbacks import EarlyStopping
batch_size = 256
early_stop = EarlyStopping(monitor='accuracy', min_delta=0.00001, patience=5, verbose=1)
modelo_sentimiento.fit(X_train, Y_train, epochs=50, batch_size=batch_size, verbose=1,
                      callbacks=[early_stop]);

Ahora evaluamos con los datos de test

In [ ]:
loss, precision_test = modelo_sentimiento.evaluate(X_test, Y_test)
precision_test

Ahora podemos tomar una nueva frase y predecir su polaridad

In [ ]:
nueva_frase = "Qué bonito es el amor"

In [ ]:
nueva_frase_tokenizada = tokenizer.texts_to_sequences([nueva_frase])
nueva_frase_tokenizada_pad = pad_sequences(nueva_frase_tokenizada, maxlen=42)

In [ ]:
nueva_frase_tokenizada

In [ ]:
nueva_frase_tokenizada_pad

In [ ]:
predictions = modelo_sentimiento.predict(nueva_frase_tokenizada_pad)
predicted_class = np.argmax(predictions)

In [ ]:
predicted_class

In [ ]:
polaridades_dict[3]

### Series Temporales 

Vamos a usar un dataset que contiene el número de pasajeros de avión internacionales por mes, obtenido de [este link](https://www.kaggle.com/andreazzini/international-airline-passengers).

En concreto vamos a intentar predecir el número de pasajeros de líneas aereas los últimos meses del dataset.

### Cargamos los datos

In [ ]:
import pandas as pd

pasajeros = pd.read_csv("international-airline-passengers.csv",sep=";")
pasajeros.columns = ["mes", "pasajeros"]

In [ ]:
pasajeros.head()

La variable objetivo de este modelo será predecir el volumen de pasajeros del mes siguiente. Podemos usar la función de pandas `shift` para mover la columna del número de pasajeros una posición hacia arriba.

In [ ]:
pasajeros["pasajeros_1"] = pasajeros["pasajeros"].shift(-1)

In [ ]:
pasajeros.tail()

Al hacer esto la última fila no tiene un valor para predecir, la eliminamos.

In [ ]:
pasajeros = pasajeros.drop(143)

In [ ]:
pasajeros_x = pasajeros["pasajeros"].astype(float).values
pasajeros_y = pasajeros["pasajeros_1"].astype(float).values

In [ ]:
n_periodos = len(pasajeros)
pct_test = 0.2

n_train = int(n_periodos * (1-pct_test))
n_train

Como siempre, estandarizamos los datos

In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
scaler.fit(pasajeros_x.reshape(-1, 1))

pasajeros_x_std = scaler.transform(pasajeros_x.reshape(-1, 1))
pasajeros_y_std = scaler.transform(pasajeros_y.reshape(-1, 1))

In [ ]:
x_train = pasajeros_x_std[:n_train]
x_test = pasajeros_x_std[n_train:]

y_train = pasajeros_y_std[:n_train]
y_test = pasajeros_y_std[n_train:]

In [ ]:
x_train = x_train.reshape(-1,1,1)
x_test = x_test.reshape(-1,1,1)

In [ ]:
x_train.shape

In [ ]:
modelo_lstm = Sequential()
modelo_lstm.add(GRU(32, return_sequences = True,input_shape=(1, 1)))
modelo_lstm.add(GRU(32,return_sequences = True))
modelo_lstm.add(GRU(32))
modelo_lstm.add(Dense(1))
modelo_lstm.summary()

In [ ]:
modelo_lstm.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
modelo_lstm.fit(x_train, y_train, epochs=100, batch_size=1, verbose=1);

In [ ]:
from sklearn.metrics import mean_squared_error
train_pred = scaler.inverse_transform(modelo_lstm.predict(x_train))
y_train_original = scaler.inverse_transform(y_train)
error_train = np.sqrt(mean_squared_error(y_train_original, train_pred))

error_train

In [ ]:
from sklearn.metrics import mean_squared_error

test_pred = scaler.inverse_transform(modelo_lstm.predict(x_test))
y_test_original = scaler.inverse_transform(y_test)
error_test = np.sqrt(mean_squared_error(y_test_original, test_pred))

error_test

In [ ]:
test_pred.shape

In [ ]:
test_pred_plot = np.zeros(pasajeros_y.shape)
test_pred_plot[-test_pred.shape[0]:] = test_pred[:,0]

test_pred_plot[:-test_pred.shape[0]] = np.nan

In [ ]:
test_pred_plot

In [ ]:
plt.plot(pasajeros_y)
plt.plot(train_pred, label="predicción train")
plt.plot(test_pred_plot, label="predicción test")
plt.title("Número de pasajeros internacionales")
plt.legend();